## Exercise: Using LSTMs to Classify the 20 Newsgroups Data Set
The 20 Newsgroups data set is a well known classification problem. The goal is to classify which newsgroup a particular post came from.  The 20 possible groups are:

`comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x	rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
misc.forsale
talk.politics.misc
talk.politics.guns
talk.politics.mideast
talk.religion.misc
alt.atheism
soc.religion.christian`

As you can see, some pairs of groups may be quite similar while others are very different.

The data is given as a designated training set of size 11314 and test set of size 7532.  The 20 categories are represented in roughly equal proportions, so the baseline accuracy is around 5%.


To begin, review the code below.  This will walk you through the basics of loading in the 20 newsgroups data, loading in the GloVe data, building the word embedding matrix, and building the LSTM model.

After we build the first LSTM model, it will be your turn to build one and play with the parameters.

In [ ]:
import numpy as np

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM

import keras

from sklearn.datasets import fetch_20newsgroups

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
max_features = 20000
seq_length = 30  # How long to make our word sequences
batch_size = 32



In [ ]:
# Download the 20 newsgroups data - there is already a designated "train" and "test" set

newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')



In [ ]:
len(newsgroups_train.data), len(newsgroups_test.data)

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(newsgroups_train.data)

In [ ]:
sequences_train = tokenizer.texts_to_sequences(newsgroups_train.data)
sequences_test = tokenizer.texts_to_sequences(newsgroups_test.data)

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 134142 unique tokens.


In [ ]:
x_train = pad_sequences(sequences_train, maxlen=seq_length)
x_test = pad_sequences(sequences_test, maxlen=seq_length)



In [ ]:
x_train

array([[ 2908,   198,     3, ...,    35,    58,  7860],
       [  351,   138,   533, ...,   118,   441,    15],
       [    9,    33,     4, ...,   187,    84, 17015],
       ...,
       [   10,     1,  1787, ...,   349,   383,    31],
       [  115,   362,    67, ...,  7772,   486,   492],
       [ 4485, 13919,  1031, ...,   200,    38,  3826]])

In [ ]:
y_train = keras.utils.to_categorical(np.asarray(newsgroups_train.target))
y_test = keras.utils.to_categorical(np.asarray(newsgroups_test.target))


We will be using the Glove pre-trained word vectors.  If you haven't already, please download them using this link:
(NOTE: this will start downloading an 822MB file)

http://nlp.stanford.edu/data/glove.6B.zip

Then unzip the file and fill your local path to the file in the code cell below.

We will use the file `glove.6B.100d.txt`

In [ ]:
embeddings_index = {}
f = open('glove.6B.100d.txt','r', encoding="utf-8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


Let's just look at a word embedding

In [ ]:
dog_vec = embeddings_index['dog']
dog_vec

In [ ]:
## This creates a matrix where the $i$th row gives the word embedding for the word represented by integer $i$.
## Essentially, these will be the "weights" for the Embedding Layer
## Rather than learning the weights, we will use these ones and "freeze" the layer

embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(134143, 100)

## LSTM Layer
`keras.layers.recurrent.LSTM(units, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- Similar in structure to the `SimpleRNN` layer
- `units` defines the dimension of the recurrent state
- `recurrent_...` refers the recurrent state aspects of the LSTM
- `kernel_...` refers to the transformations done on the input



In [ ]:
word_dimension = 100  # This is the dimension of the words we are using from GloVe
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                            word_dimension,
                            weights=[embedding_matrix],  # We set the weights to be the word vectors from GloVe
                            input_length=seq_length,
                            trainable=False))  # By setting trainable to False, we "freeze" the word embeddings.
model.add(LSTM(30, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(20, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           13414300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                15720     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
Total params: 13,430,640
Trainable params: 16,340
Non-trainable params: 13,414,300
_________________________________________________________________


In [ ]:
rmsprop = keras.optimizers.RMSprop(lr = .002)

model.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(x_test, y_test))

Train on 11314 samples, validate on 7532 samples
Epoch 1/20
11314/11314 [==============================] - 10s 920us/step - loss: 2.7524 - accuracy: 0.1445 - val_loss: 2.4695 - val_accuracy: 0.2128
Epoch 2/20
11314/11314 [==============================] - 12s 1ms/step - loss: 2.3717 - accuracy: 0.2538 - val_loss: 2.2177 - val_accuracy: 0.2993
Epoch 3/20
11314/11314 [==============================] - 12s 1ms/step - loss: 2.1776 - accuracy: 0.3204 - val_loss: 2.1056 - val_accuracy: 0.3427
Epoch 4/20
11314/11314 [==============================] - 9s 796us/step - loss: 2.0327 - accuracy: 0.3770 - val_loss: 1.9933 - val_accuracy: 0.3749
Epoch 5/20
11314/11314 [==============================] - 10s 868us/step - loss: 1.9259 - accuracy: 0.4140 - val_loss: 1.9139 - val_accuracy: 0.4006
Epoch 6/20
11314/11314 [==============================] - 9s 773us/step - loss: 1.8537 - accuracy: 0.4291 - val_loss: 1.8770 - val_accuracy: 0.4226
Epoch 7/20
11314/11314 [==============================] - 9s 78

## Exercise
### Your Turn
- Build a neural network with a SimpleRNN instead of an LSTM (with other dimensions and parameters the same). How does the performance compare?
- Use the LSTM above without the pretrained word vectors (randomly initialize the weights and have them be learned during the training process).  How does the performance compare?
- Try different sequence lengths, and dimensions for the hidden state of the LSTM.  Can you improve the model?


In [ ]:
from keras.layers import SimpleRNN
from keras import initializers


In [ ]:
rnn_hidden_dim=50
model_2 = Sequential()
model_2.add(Embedding(len(word_index) + 1,
                            100,
                            weights=[embedding_matrix],
                            input_length=seq_length,
                            trainable=False))
model_2.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu'))

model_2.add(Dense(20, activation='softmax'))

model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 100)           13414300  
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 50)                7550      
_________________________________________________________________
dense_2 (Dense)              (None, 20)                1020      
Total params: 13,422,870
Trainable params: 8,570
Non-trainable params: 13,414,300
_________________________________________________________________


In [ ]:
rmsprop = keras.optimizers.RMSprop(lr = .0002)

model_2.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_2.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=100,
          validation_data=(x_test, y_test))

Train on 11314 samples, validate on 7532 samples
Epoch 1/100
11314/11314 [==============================] - 4s 355us/step - loss: 2.5996 - accuracy: 0.1918 - val_loss: 2.4570 - val_accuracy: 0.2286
Epoch 2/100
11314/11314 [==============================] - 4s 381us/step - loss: 2.2602 - accuracy: 0.2949 - val_loss: 2.2247 - val_accuracy: 0.3093
Epoch 3/100
11314/11314 [==============================] - 4s 356us/step - loss: 2.1263 - accuracy: 0.3429 - val_loss: 2.1871 - val_accuracy: 0.3209
Epoch 4/100
11314/11314 [==============================] - 4s 338us/step - loss: 2.0366 - accuracy: 0.3743 - val_loss: 2.1642 - val_accuracy: 0.3283
Epoch 5/100
11314/11314 [==============================] - 4s 330us/step - loss: 1.9879 - accuracy: 0.3848 - val_loss: 2.0815 - val_accuracy: 0.3530
Epoch 6/100
11314/11314 [==============================] - 5s 411us/step - loss: 1.9351 - accuracy: 0.4081 - val_loss: 2.1710 - val_accuracy: 0.3405
Epoch 7/100
11314/11314 [==============================] 

In [ ]:
model_3 = Sequential()
model_3.add(Embedding(len(word_index) + 1,
                            word_dimension))  # By setting trainable to False, we "freeze" the word embeddings.
model_3.add(LSTM(30, dropout=0.2, recurrent_dropout=0.2))
model_3.add(Dense(20, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           13414300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                15720     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
Total params: 13,430,640
Trainable params: 16,340
Non-trainable params: 13,414,300
_________________________________________________________________


In [ ]:
rmsprop = keras.optimizers.RMSprop(lr = .002)

model_3.compile(loss='categorical_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [ ]:
model_3.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=20,
          validation_data=(x_test, y_test))